In [14]:
import pandas as pd
from rdkit import Chem
import requests
#from helpers.integration_helpers import canoicalize_smiles

## Carroll

In [ ]:
data = pd.read_csv('../data/Carroll11.csv', encoding = "ISO-8859-1")

In [ ]:
carroll = data[['Compound', 'TFP(K)Ref.']]
carroll['source'] = 'carroll11'
carroll['pure substance'] = 1

In [ ]:
carroll['smiles'] = carroll['Compound'].apply(getSmiles)

In [ ]:
carroll.dropna(subset=['smiles'], inplace=True)

In [ ]:
carroll.dropna(subset=['Compound'], inplace=True)

In [ ]:
carroll.head()

In [ ]:
carroll.rename(index=str, columns={"Compound": "compound", "TFP(K)Ref.": "flashpoint"}, inplace=True)

In [ ]:
carroll = carroll[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]
carroll.head()
carroll.to_csv('Carroll11.csv', index=False)

In [ ]:
print(carroll['flashpoint'].mean())
print(carroll['flashpoint'].std())

## Saldana

In [ ]:
data = pd.read_csv('../data/Saldana11(fuels).csv', encoding = "ISO-8859-1")

In [ ]:
fuels = data[['Name', 'Family', 'SMILES', 'FP Exp.']]
fuels['source'] = 'Saldana11'
fuels['pure substance'] = 1
fuels.rename(index=str, columns={"Name": "compound", "FP Exp.": "flashpoint", "SMILES": "smiles"}, inplace=True)
fuels = fuels[['smiles', 'compound', 'flashpoint', 'source', 'pure substance', 'Family']]
fuels.head()

In [ ]:
fuels.to_csv('fuels.csv', index=False)

In [ ]:
print(fuels['flashpoint'].mean())
print(fuels['flashpoint'].std())

## Pan12

In [ ]:
data = pd.read_csv('../data/pan12.csv', encoding = "ISO-8859-1")

In [ ]:
pan12 = data[['Compounds Name','smiles', 'Observed']]
pan12.rename(index=str,columns={"Compounds Name": "compound", 'Observed': 'flashpoint'}, inplace=True)
pan12['flashpoint'] = pan12['flashpoint'].apply(CtoK)
pan12['source'] = 'pan12'
pan12['pure substance'] = 1
pan12 = pan12[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]
pan12.head()

In [ ]:
pan12.to_csv('pan12.csv', index=False)

## chen14

In [ ]:
data = pd.read_csv('../data/chen14.csv', encoding = "ISO-8859-1")

In [ ]:
data.head()

In [ ]:
chen14 = data[['Compond Name','smiles', 'Observed']]
chen14.rename(index=str,columns={"Compond Name": "compound", 'Observed': 'flashpoint'}, inplace=True)
chen14['flashpoint'] = chen14['flashpoint'].apply(CtoK)
chen14['source'] = 'chen14'
chen14['pure substance'] = 1
chen14 = chen14[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]
chen14.head()

In [ ]:
chen14.to_csv('chen14.csv', index=False)

In [ ]:
carroll = pd.read_csv('Carroll11.csv', encoding = "ISO-8859-1")
fuels = pd.read_csv('fuels.csv', encoding = "ISO-8859-1")
pan12 = pd.read_csv('pan12.csv', encoding = "ISO-8859-1")
chen14 = pd.read_csv('chen14.csv', encoding = "ISO-8859-1")

In [ ]:
# canonicalize carroll smiles
for i in range(0, len(carroll)):
    carroll.iloc[i]['smiles'] = Chem.MolToSmiles(Chem.MolFromSmiles(carroll.iloc[i]['smiles']))


In [ ]:
fuels.drop(['Family'], axis=1, inplace=True)
print(carroll.columns)
print(fuels.columns)
print(pan12.columns)
print(chen14.columns)

In [ ]:
carroll['smiles']

In [ ]:
frames = [fuels, carroll, pan12, chen14]
result = pd.concat(frames)

In [ ]:
result.shape

In [ ]:
#canonicalize smiles with rdkit
result['canonical smiles'] = None
for idx, row in result.iterrows():
    result.loc[idx,'canonical smiles'] = Chem.MolToSmiles(Chem.MolFromSmiles(result.iloc[idx]['smiles']))

In [ ]:
result.dropna(inplace=True)

In [ ]:
result.shape

In [ ]:
result.to_csv('integrated_dataset.csv', index=False)

In [ ]:
result = pd.read_csv('../data/integrated_dataset.csv')

In [ ]:
result['canonical smiles'] = None
result.head(5)

In [ ]:
for idx, row in result.iterrows():
    result.loc[idx,'canonical smiles'] = Chem.MolToSmiles(Chem.MolFromSmiles(result.loc[idx]['smiles']))

In [ ]:
result.to_csv('../data/integrated_dataset.csv', index=False)

## Add Gelest to dataset

In [ ]:
#result = pd.read_csv('../data/integrated_dataset.csv')
result.drop(columns=['smiles'], inplace=True)
result.rename(index=str, columns={"canonical smiles" : "smiles"}, inplace=True)

In [ ]:
silanes = pd.read_csv('../data/silanes_all.csv')
metal_organic = pd.read_csv('../data/metal-organics_all.csv')
tin = pd.read_csv('../data/tin_all.csv')
germanium  = pd.read_csv('../data/germanium_all.csv')

In [ ]:
result = result[['compound', 'smiles', 'flashpoint', 'pure substance', 'source']]
result.head()

In [ ]:
frames = [silanes, metal_organic, tin, germanium, result]
resultv2 = pd.concat(frames)

In [ ]:
resultv2.shape
resultv2.to_csv('../data/integrated_dataset.csv')

## pubchem integration

In [ ]:
pubchem = pd.read_csv('../data/pubchem_data.csv')
carroll = pd.read_csv('../data/Carroll11.csv')
chen14 = pd.read_csv('../data/chen14.csv')
fuels = pd.read_csv('../data/fuels.csv')
germanium = pd.read_csv('../data/germanium_all.csv')
metal_organics = pd.read_csv('../data/metal-organics_all.csv')
pan12 = pd.read_csv('../data/pan12.csv')
silanes = pd.read_csv('../data/silanes_all.csv')
tin = pd.read_csv('../data/tin_all.csv')

In [ ]:
print(list(pubchem))
print(list(carroll))
print(list(chen14))
print(list(fuels))
print(list(germanium))
print(list(metal_organics))
print(list(pan12))
print(list(silanes))
print(list(tin))

In [ ]:
def canoicalize_smiles(data):
    for idx, row in data.iterrows():
        m = Chem.MolFromSmiles(data.iloc[idx]['smiles'])
        if m != None:
            data.iloc[idx]['smiles'] = Chem.MolToSmiles(m)
        else:
            data.iloc[idx]['smiles'] = None
    return data

In [ ]:
frames = [pubchem, carroll, chen14, fuels, germanium, metal_organics, pan12, silanes, tin]
result = pd.concat(frames)

In [ ]:
canonical_smiles_data = canoicalize_smiles(result)

In [ ]:
canonical_smiles_data[canonical_smiles_data.duplicated()].shape

In [ ]:
canonical_smiles_data.to_csv('../data/integrated_dataset.csv', index=False)

In [ ]:
data = canonical_smiles_data

In [ ]:
data.drop_duplicates().shape

In [ ]:
#data.drop_duplicates('smiles').
rows = data.loc[data['smiles'] == 'C[Sn](C)(C)Cl']
rows.iloc[0]#['flashpoint'].std()

In [59]:
def remove_duplicates(data):
    duplicates = data[data.duplicated(subset='smiles')]
    result = data.drop_duplicates(subset='smiles', keep=False)#[~duplicates]
    #for each unique smiles that has duplicates
    for smiles in data[data.duplicated(subset='smiles')]['smiles'].unique():
        dup_rows = data.loc[data['smiles'] == smiles]
        if dup_rows['flashpoint'].unique().shape[0] == 1:
            # remove all but one
            result = result.append(dup_rows.iloc[0], sort=False)
        else:
            if dup_rows['flashpoint'].std() < 5:
                # add 1 back
                result = result.append(dup_rows.iloc[0], sort=False)
    return result  
            

In [ ]:
pure_compounds = data.loc[data['pure substance'] == 1]
pure_compounds.shape

In [ ]:
data[data.duplicated(subset='smiles')].shape

In [ ]:
data = data.loc[data['pure substance'] == 1]
print(data.shape)
no_dups = remove_duplicates(data)
print(no_dups.shape)

In [ ]:
no_dups[no_dups.duplicated(subset='smiles')]
#no_dups.loc[no_dups['smiles'] == 'CC1=CCCC(=C)C2CC(C2CC1)(C)C']
no_dups.to_csv('../data/integrated_dataset.csv', index=False)

In [ ]:
result = data.drop_duplicates(subset='smiles', keep=False)
result.shape

In [ ]:
data[data.duplicated(subset='smiles')]['smiles'].unique()

### Hazardous Materials

In [40]:
data = pd.read_csv('../data/hazardous_materials.csv')
haz = data[['Common name', 'Data type', 'Measurement name', 'Measurement value', 'Measurement units']]

In [41]:
flashpoints = haz.loc[haz['Measurement name'] == 'Flash point']

In [42]:
print(list(flashpoints))

['Common name', 'Data type', 'Measurement name', 'Measurement value', 'Measurement units']


In [43]:
flashpoints['smiles'] = flashpoints['Common name'].apply(get_smiles)

C:\Users\Nathaniel\Miniconda3\envs\moleprop\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
def get_compound_with_element(data, element):
    return data[data['smiles'].str.contains(element)]

def get_smiles(compound):
    smilesUrl = "https://opsin.ch.cam.ac.uk/opsin/"+compound+".smi"
    requestSmiles = requests.get(smilesUrl)
    if requestSmiles.status_code == 400 or requestSmiles.status_code == 404:
        return None
    else:
        return Chem.MolToSmiles(Chem.MolFromSmiles(requestSmiles.text))  
    
def celsiusToKelvin(temp):
    return temp + 273.15

def get_compound_with_element(data, element):
    return data[data['smiles'].str.contains(element)]

In [18]:
flashpoints = flashpoints[~flashpoints['Measurement value'].str.contains('<')]
flashpoints = flashpoints[~flashpoints['Measurement value'].str.contains('>')]
flashpoints = flashpoints[~flashpoints['Measurement value'].str.contains('~')]
pattern = '\\w+-\\w'
flashpoints['Measurement value'] = flashpoints['Measurement value'].str.replace(pattern, 'none', regex=True)
#flashpoints = flashpoints[~flashpoints['Measurement value'].str.contains('none')]
flashpoints['Measurement value'] = pd.to_numeric(flashpoints['Measurement value'], errors='coerce')

flashpoints.head()

NameError: name 'flashpoints' is not defined

In [46]:
flashpoints['flashpoint'] = flashpoints['Measurement value'].apply(celsiusToKelvin)
flashpoints.drop(columns=['Measurement name','Data type', 'Measurement units', 'Measurement value'], inplace=True)
#flashpoints.rename(columns={'Common name' : 'compound', 'Measurement value' : 'flashpoint'}, inplace=True)

In [62]:
flashpoints.dropna(inplace=True)
flashpoints.rename(columns={'Common name' : 'compound'}, inplace=True)
#flashpoints.drop(df.columns[i], axis=1)
flashpoints['pure substance'] = 1
flashpoints['source'] = 'hazardous materials'
hazardous_mat = flashpoints[['compound', 'flashpoint', 'pure substance', 'smiles', 'source']]
hazardous_mat.shape

(368, 5)

In [61]:
#data = pd.read_csv('../data/integrated_dataset.csv')
#frames = [data, hazardous_mat]
#result = pd.concat(frames)
no_dups = remove_duplicates(result)
print(result.shape)
print(no_dups.shape)
no_dups.to_csv('../data/integrated_dataset.csv', index=False)

(3684, 5)
(3471, 5)


In [13]:
dataset = pd.read_csv('../data/integrated_dataset.csv')
Si_data = get_compound_with_element(dataset, 'Si')
Si_data = Si_data[['smiles', 'flashpoint']]
print(Si_data.shape)
Si_data.to_csv('../data/sidata.csv', index=False)
data = dataset[~dataset['smiles'].str.contains('Si')]
data = data[['smiles', 'flashpoint']]
data.to_csv('../data/notSidata.csv')
print(data.shape)


(298, 2)
(3173, 2)


In [ ]:
data = dataset[['smiles', 'flashpoint']]
data.to_csv('../data/flashpoints.csv')
dataset['source'].unique()

In [ ]:
data = dataset[dataset['source'] != 'gelest_metal-organics']
data = data[['smiles', 'flashpoint']]
data.to_csv('../data/flashpointsNoGelestMetallics.csv', index=False)

In [3]:
data = pd.read_csv('../data/integrated_dataset.csv')
print(data['flashpoint'].mean())
print(data['flashpoint'].std())

341.2367156439067
65.30367258221965


In [7]:
carroll = data[data['source'] == 'carroll11']
carroll = carroll[['smiles', 'flashpoint']]
carroll.head()
carroll.to_csv('../data/carroll.csv', index=False)

In [8]:
carroll = data[data['source'] == 'pubchem']
carroll = carroll[['smiles', 'flashpoint']]
carroll.head()
carroll.to_csv('../data/pubchem.csv', index=False)

In [17]:
pubchem = pd.read_csv('../data/carroll.csv')
print(pubchem['flashpoint'].mean())
print(pubchem['flashpoint'].std())

312.92808988764045
40.81731637194481
